本baseline使用的是yoloV5基本框架，推荐使用GPU进行训练，如果没有GPU加速，可以尝试去kaggle上面的kernel使用免费GPU

# 数据预处理

In [ ]:
根据yoloV5的要求，将xml文件，转化为txt文件，同时将具体坐标转换为对应的比例。

In [ ]:
from xml.dom.minidom import parse
import xml.dom.minidom
import os

In [ ]:
l=['knife',
 'scissors',
 'sharpTools',
 'expandableBaton',
 'smallGlassBottle',
 'electricBaton',
 'plasticBeverageBottle',
 'plasticBottleWithaNozzle',
 'electronicEquipment',
 'battery',
 'seal',
 'umbrella']
dic = {}

In [ ]:
for n,i in enumerate(l):
    dic[i] = str(n)

In [ ]:
path = ".\\初赛第一阶段_训练集\\domain1\\XML\\"
path2 = ".\\初赛第一阶段_训练集\\domain1\\label\\"
# 写入path2对应的文件夹

In [ ]:
for i in os.listdir(path):
    l0 = []
    DOMTree = xml.dom.minidom.parse(path+i)
    root = DOMTree.documentElement
    
    width = int(root.getElementsByTagName('size')[0].getElementsByTagName('width')[0].firstChild.data)
    height = int(root.getElementsByTagName('size')[0].getElementsByTagName('height')[0].firstChild.data)
    for j in root.getElementsByTagName('object'):
        l1 = []
        l1.append(dic[j.getElementsByTagName('name')[0].firstChild.data])
        xmin = int(j.getElementsByTagName('bndbox')[0].getElementsByTagName('xmin')[0].firstChild.data)
        ymin = int(j.getElementsByTagName('bndbox')[0].getElementsByTagName('ymin')[0].firstChild.data)
        xmax = int(j.getElementsByTagName('bndbox')[0].getElementsByTagName('xmax')[0].firstChild.data)
        ymax = int(j.getElementsByTagName('bndbox')[0].getElementsByTagName('ymax')[0].firstChild.data)
        l1.append(str((xmin+xmax)/2/width))
        l1.append(str((ymin+ymax)/2/height))
        l1.append(str((xmax-xmin)/width))
        l1.append(str((ymax-ymin)/height))
        l0.append(l1)
    with open(path2 + i.split('.')[0]+'.txt','w',encoding='utf-8') as f:
        [f.write('{0}\n'.format(' '.join(value))) for value in l0]

# 训练

## kaggle或linux版本

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.7.0 (CPU)


In [3]:
# Weights & Biases  (optional)
%pip install -q wandb
import wandb
wandb.login()

In [ ]:
!python train.py --img 640 --batch 32 --epochs 100 --data /kaggle/input/findxray/kaggle.yaml --weights /kaggle/input/findxray/yolov5s.pt --nosave --cache
# 其中，kaggle.yaml和yolov5s.pt文件的目录需要根据自己情况修改。最终结果输出到runs/train/exp文件夹下

In [ ]:
!python detect.py --weights ./runs/train/exp/weights/last.pt --img 640 --conf 0.5 --source ../test/ --save-txt  --save-conf
# 其中，last.pt和test文件夹的目录需要根据自己情况修改

## windows版本

请参考
https://github.com/wudashuo/yolov5
https://blog.csdn.net/qq_45701791/article/details/113992622
下载依赖包，下载yoloV5代码，之后在powershell（命令行）当中运行训练及测试命令即可。

# 输出结果

In [ ]:
import json
import os
import pandas as pd
import cv2

In [ ]:
# 读取图片大小
df_size = pd.DataFrame(columns=['name','width','height'])

path = 'E:\\xray\\test\\'
for file in os.listdir(path):
    img = cv2.imread(path+file)
    size = img.shape
    df_size.loc[df_size.shape[0]] = [file.split('.')[0],size[1],size[0]]
    
df_size.to_csv('size.csv',index=False)

In [ ]:
df_size = pd.read_csv('size.csv')
df_size =  df_size.set_index('name')
df_size

In [ ]:
# 因为yoloV5最终预测的是比例，需要根据图像大小把比例还原成坐标

result=[]
path = 'E:\\xray\\yolov5-master\\runs\\detect\\exp\\labels\\'
path2 = 'E:\\xray\\test\\'
for file in os.listdir(path2):
    pic = []
    try:
        df = pd.read_csv(path+file.split('.')[0]+'.txt',header=None,delimiter=' ')
        df[0] = df[0]+1
        df = df.set_index(0)

        df[1] = df[1]*df_size.loc[file.split('.')[0],'width']
        df[3] = df[3]*df_size.loc[file.split('.')[0],'width']

        df[2] = df[2]*df_size.loc[file.split('.')[0],'height']
        df[4] = df[4]*df_size.loc[file.split('.')[0],'height']
        
        df[1],df[2],df[3],df[4] = df[1]-df[3]/2,df[2]-df[4]/2,df[1]+df[3]/2,df[2]+df[4]/2
        for i in range(1,13):
            try:
                pic.append(df.loc[i].values.tolist())
            except:
                pic.append([])
    except:
        for i in range(1,13):
            pic.append([])
    result.append(pic)

In [ ]:
# 输出文件
with open('sub.json','w') as f:
    f.write(str(result))